In [1]:
import numpy as np
import cupy as cp
import pandas as pd
import matplotlib.pyplot as plt

from ipywidgets import interact

from ipywidgets import widgets
from tqdm.auto import tqdm

from vol.vol import Heston

from hestonmc import MarketState, HestonParameters, mc_price, simulate_heston_euler, simulate_heston_andersen_qe, simulate_heston_andersen_tg
from hestonmc_cuda import mc_price_cupy, simulate_heston_euler_cupy, simulate_heston_andersen_qe_cupy, simulate_heston_andersen_tg_cupy, european_call_payoff_cupy
from derivatives import european_call_payoff

In [2]:
heston_params_1 = HestonParameters(kappa = 1.3125, gamma = 0.5125, rho = -0.3937, vbar = 0.0641, v0 = 0.3) #from stoch vol
heston_params_2 = HestonParameters(kappa = 1, gamma = 0.4, rho = -0.1, vbar = 0.2, v0 = 0.2) # from school
heston_params_3 = HestonParameters(kappa = 0.5, gamma = 1, rho = -0.9, vbar = 0.04, v0 = 0.04) #  from andeson paper 1
heston_params_4 = HestonParameters(kappa = 0.3, gamma = 0.9, rho = -0.5, vbar = 0.04, v0 = 0.04) #  from andeson paper 2
heston_params_5 = HestonParameters(kappa = 1, gamma = 1, rho = -0.3, vbar = 0.04, v0 = 0.09) #  from andeson paper 3

heston_params_array = [heston_params_1, heston_params_2, heston_params_3, heston_params_4, heston_params_5]

state         = MarketState(stock_price = 1.*100, interest_rate = 0.)

r_x           = np.load(r"Data/anderson tg/r_x start=1e-07 stop=100 N=4999998 dt=2e-05.npy")
f_nu_y        = np.load(r"Data/anderson tg/f_nu_y start=1e-07 stop=100 N=4999998 dt=2e-05.npy")
f_sigma_y     = np.load(r"Data/anderson tg/f_sigma_y start=1e-07 stop=100 N=4999998 dt=2e-05.npy")
kwargs        = {'x_grid' : r_x, 'f_nu_grid' : f_nu_y, 'f_sigma_grid' : f_sigma_y }
r_x           = cp.load(r"Data/anderson tg/r_x start=1e-07 stop=100 N=4999998 dt=2e-05.npy")
f_nu_y        = cp.load(r"Data/anderson tg/f_nu_y start=1e-07 stop=100 N=4999998 dt=2e-05.npy")
f_sigma_y     = cp.load(r"Data/anderson tg/f_sigma_y start=1e-07 stop=100 N=4999998 dt=2e-05.npy")
kwargs_cupy        = {'x_grid' : r_x, 'f_nu_grid' : f_nu_y, 'f_sigma_grid' : f_sigma_y }

In [3]:
import time
from tqdm.contrib.itertools import product

In [4]:
model = Heston(state.stock_price, heston_params_3.v0, heston_params_3.kappa, heston_params_3.vbar, heston_params_3.gamma, heston_params_3.rho, state.interest_rate)
model.call_price(0.4, 70)

30.428646036927244

In [ ]:
test_1 = {'schemes':[simulate_heston_euler_cupy, simulate_heston_andersen_tg_cupy],
          'strikes': np.linspace(70, 120, 70),
            'Ts': np.linspace(0.1, 5, 70),
            'N_Ts': range(50, 125, 50),
            'batch_sizes': range(100_000, 1000_000, 100000_000),
            'heston_params': heston_params_1
          }

MC_compare_models_grid_test_1= pd.DataFrame(columns=['scheme' ,'heston_params#', 'strike', 'T', 'N_T', 'batch_size', 'absolute error', 'true' , 'MC_price', 'time'])

In [19]:
test_2 = {'schemes':[simulate_heston_euler_cupy, simulate_heston_andersen_tg_cupy],
          'strikes': np.linspace(70, 120, 20),
            'Ts': np.linspace(0.1, 5, 20),
            'N_Ts': range(50, 125, 500),
            'batch_sizes': range(100_000, 1000_000, 100000_000),
            'heston_params_n': [1, 2, 3, 4, 5]

        }

MC_compare_models_grid_test_2= pd.DataFrame(columns=['scheme' ,'heston_params#', 'strike', 'T', 'N_T', 'batch_size', 'absolute error', 'true' , 'MC_price', 'time'])

In [5]:
test_3 = {'schemes':[simulate_heston_euler_cupy],
          'strikes': np.linspace(60, 140, 30),
            'Ts': np.linspace(0.1, 5, 30),
            'N_Ts': range(50, 125, 50),
            'batch_sizes': range(500_000, 1000_000, 100000_000),
            'heston_params_n': [1, 2, 3, 4, 5]

        }

MC_compare_models_grid_test_3_euler= pd.DataFrame(columns=['scheme' ,'heston_params#', 'strike', 'T', 'N_T', 'batch_size', 'absolute error', 'true' , 'MC_price', 'time'])

In [ ]:
test_3 = {'schemes':[simulate_heston_andersen_tg_cupy],
          'strikes': np.linspace(60, 140, 30),
            'Ts': np.linspace(0.1, 5, 30),
            'N_Ts': range(50, 125, 50),
            'batch_sizes': range(500_000, 1000_000, 100000_000),
            'heston_params_n': [1, 2, 3, 4, 5]

        }

MC_compare_models_grid_test_3_andersen= pd.DataFrame(columns=['scheme' ,'heston_params#', 'strike', 'T', 'N_T', 'batch_size', 'absolute error', 'true' , 'MC_price', 'time'])

In [ ]:
test_3 = {'schemes':[simulate_heston_euler_cupy, simulate_heston_andersen_tg_cupy],
          'strikes': np.linspace(60, 140, 30),
            'Ts': np.linspace(0.1, 5, 30),
            'N_Ts': range(25, 180, 50),
            'batch_sizes': range(500_000, 1000_000, 100000_000),
            'heston_params_n': [1, 2, 3, 4, 5]

        }

MC_compare_models_grid_test_3_andersen= pd.DataFrame(columns=['scheme' ,'heston_params#', 'strike', 'T', 'N_T', 'batch_size', 'absolute error', 'true' , 'MC_price', 'time'])

In [10]:
test_3 = {'schemes':[simulate_heston_andersen_qe_cupy],
          'strikes': np.linspace(60, 140, 30),
            'Ts': np.linspace(0.1, 5, 30),
            'N_Ts': range(50, 125, 50),
            'batch_sizes': range(500_000, 1000_000, 100000_000),
            'heston_params_n': [1, 2, 3, 4, 5]

        }

MC_compare_models_grid_test_3_qe= pd.DataFrame(columns=['scheme' ,'heston_params#', 'strike', 'T', 'N_T', 'batch_size', 'absolute error', 'true' , 'MC_price', 'time'])

In [5]:
test_3 = {'schemes':[simulate_heston_andersen_qe_cupy],
          'strikes': np.linspace(60, 140, 30),
            'Ts': np.linspace(0.1, 5, 30),
            'N_Ts': range(50, 125, 50),
            'batch_sizes': range(500_000, 1000_000, 100000_000),
            'heston_params_n': [1, 2, 3, 4, 5]

        }

MC_compare_models_grid_test_3_qe= pd.DataFrame(columns=['scheme' ,'heston_params#', 'strike', 'T', 'N_T', 'batch_size', 'absolute error', 'true' , 'MC_price', 'time'])

In [6]:

i = 0

for scheme, strike, T, N_T, batch_size , heston_params_n in product(test_3['schemes'], test_3['strikes'], test_3['Ts'], test_3['N_Ts'], test_3['batch_sizes'], test_3['heston_params_n']):
    heston_params_ = heston_params_array[heston_params_n-1]
    ec_payoff = european_call_payoff_cupy(T, strike, state.interest_rate)
    
    common_mc_params = {"absolute_error": 5e-2, "state": state, "heston_params": heston_params_, "payoff": ec_payoff, "T": T, "random_seed": 42, "verbose": False}
    model = Heston(state.stock_price, heston_params_.v0, heston_params_.kappa, heston_params_.vbar, heston_params_.gamma, heston_params_.rho, state.interest_rate)
    
    if scheme == simulate_heston_andersen_tg_cupy:
        st = time.time()
        res = float(mc_price_cupy(N_T = N_T, simulate = scheme, batch_size=batch_size, **common_mc_params, **kwargs_cupy))
        et = time.time()
    elif scheme == simulate_heston_euler_cupy:
        st = time.time()
        res = float(mc_price_cupy(N_T = N_T, simulate = scheme, batch_size=batch_size, **common_mc_params))
        et = time.time()
    elif scheme == simulate_heston_andersen_qe_cupy:
        st = time.time()
        res = float(mc_price_cupy(N_T = N_T, simulate = scheme, batch_size=batch_size, **common_mc_params))
        et = time.time()
    MC_compare_models_grid_test_3_qe.loc[i] = (scheme.__name__, heston_params_n, strike, T, N_T, batch_size,common_mc_params['absolute_error'], model.call_price(T, strike), res, et-st)
    i +=1

  0%|          | 0/9000 [00:00<?, ?it/s]

/media/kir1200/E4C4BDFBC4BDCFD2/Other/code/jypeter/Stochastic-Volatility-Models/Code/vol/vol/heston.py:96: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  intg.quad(
/media/kir1200/E4C4BDFBC4BDCFD2/Other/code/jypeter/Stochastic-Volatility-Models/Code/vol/vol/heston.py:96: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probab

KeyboardInterrupt: 

In [10]:
MC_compare_models_grid_test_3_qe.to_csv(r"Data/evaluation/MC_compare_models_grid_test_3_qe.csv")

In [7]:
MC_compare_models_grid_test_3_euler.to_csv(r"Data/evaluation/MC_compare_models_grid_test_3_euler.csv")

In [76]:
MC_compare_models_grid_test_2.to_csv(r"Data/evaluation/MC_compare_models_grid_test_2.csv")